In [1]:
import os
import sys
import time
import socket
import random
import argparse
import importlib
import statistics
import scipy.misc
import numpy as np
import tensorflow as tf
from scipy import stats
import matplotlib.pyplot as plt
from timeit import default_timer as timer
%matplotlib inline

# Append python path
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'models'))

# Import pointnet stuff
import provider
import modelnet_dataset
import modelnet_h5_dataset

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# BASIC CONFIGURATION

In [3]:
BATCH_SIZE = 32
NUM_POINT = 1024
MODEL_DIR = 'logs_shapenet/log_'
GPU_INDEX = 0
MODEL = importlib.import_module('pointnet2_cls_ssg') # import network module
DUMP_DIR = 'dump'
VISUALIZATION = False
if not os.path.exists(DUMP_DIR):
    os.mkdir(DUMP_DIR)
    
HOSTNAME = socket.gethostname()
NUM_CLASSES = 55
SHAPE_NAMES = [line.rstrip() for line in \
    open(os.path.join(BASE_DIR, 'data/shapenet_core55_1024/shape_names.txt'))] 

# ShapenetCore55 train/test split
TRAIN_DATASET = modelnet_h5_dataset.ModelNetH5Dataset(os.path.join(BASE_DIR, 'data/shapenet_core55_1024/train_files.txt'), batch_size=BATCH_SIZE, npoints=NUM_POINT, shuffle=True)
TEST_DATASET = modelnet_h5_dataset.ModelNetH5Dataset(os.path.join(BASE_DIR, 'data/shapenet_core55_1024/test_files.txt'), batch_size=BATCH_SIZE, npoints=NUM_POINT, shuffle=False)

# EVALUATE ONE EPOCH METHOD

Args:

  **sess** - tf session
  
  **ops** - parameters
  
  

Returns:

  **loss** - mean classification loss
  
  **accuracy** - instance classification accuracy
  
  **class_accuracy** - class classification accuracy
  
  **predictions** - array of the output ofthe classification module with shape: (N, 40), where N is the test clouds len
  
  **true_labels** - true labels of test clouds with the lenght of N
  

In [4]:
def eval_one_epoch(sess, ops, num_votes=1, topk=1, verbose=False):
    is_training = False

    # Make sure batch data is of same size
    cur_batch_data = np.zeros((BATCH_SIZE,NUM_POINT,TEST_DATASET.num_channel()))
    cur_batch_label = np.zeros((BATCH_SIZE), dtype=np.int32)

    total_correct = 0
    total_seen = 0
    loss_sum = 0
    batch_idx = 0
    shape_ious = []
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    fout = open(os.path.join(DUMP_DIR, 'pred_label.txt'), 'w')
    pred_vals = []
    true_vals = []
    TEST_DATASET.reset()
    while TEST_DATASET.has_next_batch():
        batch_data, batch_label = TEST_DATASET.next_batch(augment=False)
        bsize = batch_data.shape[0]
        if verbose:
            print('Batch: %03d, batch size: %d'%(batch_idx, bsize))
        # for the last batch in the epoch, the bsize:end are from last batch
        cur_batch_data[0:bsize,...] = batch_data
        cur_batch_label[0:bsize] = batch_label

        batch_pred_sum = np.zeros((BATCH_SIZE, NUM_CLASSES)) # score for classes
        for vote_idx in range(num_votes):
            # Shuffle point order to achieve different farthest samplings
            shuffled_indices = np.arange(NUM_POINT)
            np.random.shuffle(shuffled_indices)
            rotated_data = provider.rotate_point_cloud_by_angle(cur_batch_data[:, shuffled_indices, :],
                    vote_idx/float(num_votes) * np.pi * 2)
            feed_dict = {ops['pointclouds_pl']: rotated_data,
                         ops['labels_pl']: cur_batch_label,
                         ops['is_training_pl']: is_training}
            loss_val, pred_val = sess.run([ops['loss'], ops['pred']], feed_dict=feed_dict)
            batch_pred_sum += pred_val
       
        # DANIEL
        pred_vals.append(batch_pred_sum[0:bsize].copy())
        true_vals.append(cur_batch_label[0:bsize].copy())
        
        # argmax
        pred_val = np.argmax(batch_pred_sum, 1)   
        correct = np.sum(pred_val[0:bsize] == batch_label[0:bsize])

        # total values
        total_correct += correct
        total_seen += bsize
        loss_sum += loss_val
        for i in range(bsize):
            l = batch_label[i]
            total_seen_class[l] += 1
            total_correct_class[l] += (pred_val[i] == l)
    
    eval_mean_loss = (loss_sum / float(total_seen))
    eval_instance_acc = (total_correct / float(total_seen))
    eval_class_acc = (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float)))
    
    pred_vals = np.concatenate(pred_vals)
    true_vals = np.concatenate(true_vals)
    
    if verbose:
        print('eval mean loss: %f' % eval_mean_loss)
        print('eval accuracy: %f' % eval_instance_acc)
        print('eval avg class acc: %f' % eval_class_acc)
    
    class_accuracies = np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float)
    if verbose:
        for i, name in enumerate(SHAPE_NAMES):
            print('%10s:\t%0.3f' % (name, class_accuracies[i]))
            
    return eval_mean_loss, eval_instance_acc, eval_class_acc, pred_vals, true_vals

# EVALUATE METHOD

Args:

  **model_num** - number of the pointnet model (i.e.: 3 to evaluate model_3.ckpt model)
  
  **num_votes** - how many votes (one vote is one pc rotation & permutation) should be used to eval the model

Returns:

  ** same as *eval_one_epoch* **

In [5]:
def evaluate(model_num, num_votes, verbose=False):
    is_training = False
    
    # Reset
    tf.reset_default_graph()
     
    with tf.device('/GPU:'+str(GPU_INDEX)):
        pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
        is_training_pl = tf.placeholder(tf.bool, shape=())

        # simple model
        pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, num_classes=NUM_CLASSES)
        MODEL.get_loss(pred, labels_pl, end_points)
        losses = tf.get_collection('losses')
        total_loss = tf.add_n(losses, name='total_loss')

        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()
        
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = True
    sess = tf.Session(config=config)

    # Get model path
    model_path = os.path.join(MODEL_DIR + str(model_num), 'model.ckpt')
    
    # Restore variables from disk.
    saver.restore(sess, model_path)
    if verbose:
        print("Model restored.")

    ops = {'pointclouds_pl': pointclouds_pl,
           'labels_pl': labels_pl,
           'is_training_pl': is_training_pl,
           'pred': pred,
           'loss': total_loss}

    return eval_one_epoch(sess, ops, num_votes, verbose)

In [6]:
start = timer()
loss, instance_acc, class_acc, pred_vals, true_vals = evaluate(model_num=0, num_votes=1, verbose=False)
end = timer()
print ('acc =', instance_acc, 'calc time [min] =', (end-start)/60)

INFO:tensorflow:Restoring parameters from logs_shapenet/log_0/model.ckpt
('acc =', 0.8954273219887822, 'calc time [min] =', 2.6002861499786376)


# # # # # # # # # # # # # # # # # # # # # # # # # # #
# NUM_VOTES TEST
# # # # # # # # # # # # # # # # # # # # # # # # # # #

In [ ]:
NUM_VOTES = 15
RANGE_MODELS = range(1, 11)
num_votes_accs = {i: [] for i in RANGE_MODELS}
for i in num_votes_accs:
    for x in range(1, NUM_VOTES+1):
        _, acc, _, _, _ = evaluate(model_num=i, num_votes=x, verbose=False)
        num_votes_accs[i].append(acc)
        print ('i=', i, 'x=', x, 'acc = ', acc)

In [ ]:
for i in num_votes_accs:
    plt.plot(np.arange(1, 1+len(num_votes_accs[i])), num_votes_accs[i])

In [ ]:
num_votes_accs_np = np.zeros((max(RANGE_MODELS), NUM_VOTES), dtype=np.float)
for i in num_votes_accs:
    num_votes_accs_np[i-1] = num_votes_accs[i]
np.save('log/num_votes_accs.npy', num_votes_accs_np)

# # # # # # # # # # # # # # # # # # # # # # # # #
# MODEL ENSEMBLING 
# # # # # # # # # # # # # # # # # # # # # # # # #

## Calculate probabilities for each test cloud and each model. The output probability array will be the shape of (N, 40, X), where N is the test cloud len and X is the models count to be ensembled.


In [7]:
NUM_MODELS = 10
NUM_VOTES = 12
probabilities = []
true_labels = []
accuracies = []
for x in range(NUM_MODELS):
    start = timer()
    loss, instance_acc, class_acc, pred_vals, true_vals = evaluate(model_num=x, num_votes=NUM_VOTES, verbose=False)
    end = timer()
    probabilities.append(pred_vals)
    accuracies.append(instance_acc)
    true_labels = np.array(true_vals)
    print ('Model =', x, 'acc =', instance_acc, 'time [min] =', (end-start)/60)

probabilities = np.stack(probabilities).transpose(1, 2, 0)
accuracies = np.array(accuracies)

np.save('logs_shapenet/probabilities.npy', probabilities)
np.save('logs_shapenet/true_labels.npy', true_labels)
np.save('logs_shapenet/accuracies.npy', accuracies)

INFO:tensorflow:Restoring parameters from logs_shapenet/log_0/model.ckpt
('Model =', 0, 'acc =', 0.8964730487688943, 'time [min] =', 16.003492951393127)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_1/model.ckpt
('Model =', 1, 'acc =', 0.8959026523433786, 'time [min] =', 15.597500018278758)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_2/model.ckpt
('Model =', 2, 'acc =', 0.8956174541306208, 'time [min] =', 15.889894898732503)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_3/model.ckpt
('Model =', 3, 'acc =', 0.8972335773362486, 'time [min] =', 16.044625119368234)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_4/model.ckpt
('Model =', 4, 'acc =', 0.8960927844852172, 'time [min] =', 16.10136084953944)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_5/model.ckpt
('Model =', 5, 'acc =', 0.8968533130525715, 'time [min] =', 16.076002816359203)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_6/model.ckpt
('Model ='

NotFoundError: logs_shapenet/log_7; No such file or directory

In [8]:
for x in [7, 8, 9]:
    start = timer()
    loss, instance_acc, class_acc, pred_vals, true_vals = evaluate(model_num=x, num_votes=NUM_VOTES, verbose=False)
    end = timer()
    probabilities.append(pred_vals)
    accuracies.append(instance_acc)
    true_labels = np.array(true_vals)
    print ('Model =', x, 'acc =', instance_acc, 'time [min] =', (end-start)/60)

probabilities = np.stack(probabilities).transpose(1, 2, 0)
accuracies = np.array(accuracies)

np.save('logs_shapenet/probabilities.npy', probabilities)
np.save('logs_shapenet/true_labels.npy', true_labels)
np.save('logs_shapenet/accuracies.npy', accuracies)

INFO:tensorflow:Restoring parameters from logs_shapenet/log_7/model.ckpt
('Model =', 7, 'acc =', 0.8987546344709573, 'time [min] =', 15.883458085854848)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_8/model.ckpt
('Model =', 8, 'acc =', 0.8974237094780873, 'time [min] =', 15.990689218044281)
INFO:tensorflow:Restoring parameters from logs_shapenet/log_9/model.ckpt
('Model =', 9, 'acc =', 0.8958075862724594, 'time [min] =', 16.11862374941508)


# Models evaluation statistics

In [9]:
print('Mean accuracy =', statistics.mean(accuracies))
indices = {}
for k in range(40):
    indices[k] = [i for i, x in enumerate(true_labels) if x == k]
    
validation_max_res = []
test_res_at_validation_max = []

for _ in range(1000):
    validation_indices = []
    test_indices = []
    for k in indices:
        random.shuffle(indices[k])
        split_idx = int(len(indices[k])/2)
        validation_indices += indices[k][:split_idx]
        test_indices += indices[k][split_idx:]
    validation_indices = sorted(validation_indices)
    test_indices = sorted(test_indices)

    validation_true_labels = true_labels[validation_indices]
    validation_probabilities = probabilities[validation_indices]
    test_true_labels = true_labels[test_indices]
    test_probabilities = probabilities[test_indices]

    validation_predictions = np.argmax(validation_probabilities, axis=1)
    validation_compare = np.equal(validation_predictions, np.expand_dims(validation_true_labels, -1))
    validation_accuracies = np.mean(validation_compare, axis=0)

    test_predictions = np.argmax(test_probabilities, axis=1)
    test_compare = np.equal(test_predictions, np.expand_dims(test_true_labels, -1))
    test_accuracies = np.mean(test_compare, axis=0)

    validation_max_res.append(np.max(validation_accuracies))
    test_res_at_validation_max.append(test_accuracies[np.argmax(validation_accuracies)])
    
mean_valid_max = statistics.mean(validation_max_res)
mean_test_at_valid_max = statistics.mean(test_res_at_validation_max)

print('Mean of validation max results =', mean_valid_max)
print('Mean of test result for validation max =', mean_test_at_valid_max)

('Mean accuracy =', 0.8967962734100199)
('Mean of validation max results =', 0.8833276018099546)
('Mean of test result for validation max =', 0.8799586578789694)


# Agregate the outputs with sum operation

In [10]:
aggregated_probability = np.sum(probabilities, axis=-1)
aggregated_predictions = np.argmax(aggregated_probability, axis=-1)
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.9004658237475045

# Agregate the outputs with mean operation

In [11]:
aggregated_probability = np.mean(probabilities, axis=-1)
aggregated_predictions = np.argmax(aggregated_probability, axis=-1)
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.9004658237475045

# Agregate the outputs with mode operation

In [12]:
aggregated_predictions = np.argmax(probabilities, axis=1)
aggregated_predictions =  np.squeeze(stats.mode(aggregated_predictions, axis=1)[0])
float(np.sum(aggregated_predictions == true_labels)) / len(true_labels)

0.899990493392908